In [6]:
import json

from google.cloud import storage
from mongodb import *
from pyspark.sql import Row, SparkSession
import pyspark
from user_definition import *


In [7]:
sc = pyspark.SparkContext()

23/02/26 16:19:50 WARN Utils: Your hostname, Ajayeswars-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.0.0.11 instead (on interface en0)
23/02/26 16:19:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/26 16:19:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
mongo_username

In [126]:
import pymongo


class MongoDBCollection:
    def __init__(self,
                 username,
                 password,
                 ip_address,
                 database_name,
                 collection_name):
        '''
        Using ip_address, database_name, collection_name,
        initiate the instance's attributes including ip_address,
        database_name, collection_name, client, db and collection.

        For pymongo, see more details in the following.
        https://pymongo.readthedocs.io
        '''
        self.username = username
        self.password = password
        
        self.ip_address = ip_address
        self.database_name = database_name
        self.collection_name = collection_name

        self.client = pymongo.MongoClient(f"mongodb+srv://{username}:{password}@{ip_address}")
        self.db = self.client[database_name]
        self.collection = self.db[collection_name]
        
        self.connection_string = f"mongodb+srv://{username}:{password}@{ip_address}/{database_name}.{collection_name}"

    def return_db(self):
        '''
        Return db which is the database in the client
        '''
        return self.db

    def return_collection(self):
        '''
        Return db which belongs to the db.
        '''
        return self.collection

    def return_num_docs(self, query):
        '''
        Return the number of documents satisfying the given query.
        '''
        return self.collection.count_documents(query)

    def drop_collection(self):
        '''
        Drop the collection
        '''
        return self.collection.drop()

    def find(self, query, projection):
        '''
        Return an iteratatable using query and projection.
        '''
        for item in self.collection.find(query, projection):
            yield item

    def insert_one(self, doc):
        '''
        Insert the given document
        '''
        self.collection.insert_one(doc)

    def insert_many(self, docs):
        '''
        Insert the given documents
        '''
        self.collection.insert_many(docs)

    def update_many(self, filter, update):
        '''
        Update documents satisfying filter with update.
        Both filter and update are dictionaries.
        '''
        self.collection.update_many(filter, update)

    def find_id_for_cad_number(self, cad_number):
        '''
        Return _id for the given cad_numer
        '''
        return self.find({"cad_number": f"{cad_number}"}, {"_id": 1})

    def remove_record_in_weather(self, cad_number, val):
        '''
        Remove items in the weather array where the "value" is val.
        '''
        return self.update_many({"cad_number": f"{cad_number}"},
                                {'$pull': {"weather": {"value": val}}})


In [4]:
def return_json(service_account_key_file,
                bucket_name,
                blob_name):
    storage_client = storage.Client.from_service_account_json('/Users/ajay/Downloads/dds/ResumeMatcher/data_collection/msds697-group6-key.json')
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    json_str = blob.download_as_string().decode("utf8")
    json_data = json.loads(json_str)
    return json_data

In [5]:
yesterday = '2023-02-22'
jobs_data_path = f'{yesterday}/jobs.json'
jobs_json = return_json(service_account_key_file,
                               bucket_name,
                               jobs_data_path)

In [7]:
jobs_json.keys()

dict_keys(['SmMip7HqlkkAAAAAAAAAAA==', '0WBOJan3AZEAAAAAAAAAAA==', 'OMY_1fMqjVoAAAAAAAAAAA==', 'uYyH_F01ClwAAAAAAAAAAA==', 'RzFaYg_dytcAAAAAAAAAAA==', 'PLjfvqTM_cEAAAAAAAAAAA==', 'vBNGSQeap8cAAAAAAAAAAA==', 'uEOSwQHXdOkAAAAAAAAAAA==', 'TMX7lnb35pwAAAAAAAAAAA==', 'SKF6g2gJELEAAAAAAAAAAA==', 'Y1gXC0-lnBYAAAAAAAAAAA==', 'BbDqzJKFKgcAAAAAAAAAAA==', 'y8az4xDo9sgAAAAAAAAAAA==', 'PGc8ikPq4EgAAAAAAAAAAA==', 'XbdDbUXF-ZAAAAAAAAAAAA==', 'Gf6t0O9mWFkAAAAAAAAAAA==', '5hBtp2DVuX0AAAAAAAAAAA==', 'dnn7rKLAtYsAAAAAAAAAAA==', 'z2vZPYOx9rsAAAAAAAAAAA=='])

In [116]:
jobs_rdd = sc.parallelize([jobs_json])
jobs_rdd.count()

1

In [117]:
def filter_fields(ip_json):
    keys = list(ip_json.keys())
    final_list = []
    for k in keys:
        job_j = ip_json[k][0]
        filtered_dict = {}
        filtered_dict['job_id'] = job_j['job_id']
        filtered_dict['job_title'] = job_j['job_title']
        filtered_dict['job_description'] = job_j['job_description']
        filtered_dict['employer_name'] = job_j['employer_name']
        filtered_dict['job_required_skills'] = job_j['job_required_skills']
        filtered_dict['job_salary_currency'] = job_j['job_salary_currency']
        filtered_dict['job_highlights'] = job_j['job_highlights']
        final_list.append(filtered_dict)
    return final_list


In [129]:
jobs_aggregate = jobs_rdd.map(lambda x:filter_fields(x))
jobs_filtered_list =jobs_aggregate.collect()[0]

In [ ]:
for aggregate in jobs_filtered_list:
    print(aggregate.keys())

# jobs_seo

In [10]:
jobs_seo_data_path = f'{yesterday}/jobs_seo.json'
jobs_seo_json = return_json(service_account_key_file,
                               bucket_name,
                               jobs_seo_data_path)

In [11]:
jobs_seo_rdd = sc.parallelize([jobs_seo_json])
jobs_seo_rdd.count()

1

In [25]:
def filter_fields_seo(ip_json):
    keys = list(ip_json.keys())
    final_list = []
    for k in keys:
        job_j = ip_json[k]
        filtered_dict = {}
        # filtered_dict['job_id'] = job_j['job_id']
        filtered_dict['job_title'] = job_j['position']
        filtered_dict['job_description'] = job_j['description']
        filtered_dict['employer_name'] = job_j['employer']
        # filtered_dict['job_required_skills'] = job_j['job_required_skills']
        # filtered_dict['job_salary_currency'] = job_j['job_salary_currency']
        filtered_dict['job_highlights'] = job_j['highlights']
        final_list.append(filtered_dict)
    return final_list


In [26]:
jobs_seo_aggregate = jobs_seo_rdd.map(lambda x:filter_fields_seo(x))
jobs_seo_filtered_list =jobs_seo_aggregate.collect()[0]

In [29]:
mongodb = MongoDBCollection(mongo_username,
                                mongo_password,
                                mongo_ip_address,
                                database_name,
                                collection_name)

for aggregate in jobs_seo_filtered_list:
    mongodb.insert_one(aggregate)